# Tutorial: Make an xarray Dataset CF-1.12 compliant

This notebook shows how to use `nc-check` to:
- inspect CF compliance with `ds.check.check_compliant()`
- apply safe fixes with `ds.check.make_cf_compliant()`

The object stays an `xarray.Dataset` throughout.


## 1. Setup


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import xarray as xr

import nc_check  # noqa: F401

## 2. Build a dataset with missing CF metadata


In [19]:
ds = xr.open_dataset(
    "/Users/luke/Downloads/GCB-2025_dataprod_CSIR-ML6_1982-2024.nc", decode_cf=False
)

## 3. Check compliance before coercion


In [ ]:
issues_before = ds.check.cf(domain="ocean", pretty_print=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ CF Compliance Report                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CF version     CF-1.12               
Engine         cfchecker             
Engine status  ok                    
Check method   cfchecker             
Conventions    cf, ferret            
Counts         fatal=0 error=8 warn=2

                                Global Findings                                 
┏━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope  ┃ Convention ┃ Severity ┃ Detail                                      ┃
┡━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ global │ cf         │ WARN     │ (2.6.1): No 'Conventions' attribute present │
└────────┴────────────┴──────────┴─────────────────────────────────────────────┘

                                                Coordinate Findings                                                
┏━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope  ┃ Convention ┃ Severity ┃ Detail                                                                         ┃
┡━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Region │ cf         │ ERROR    │ (2.2): Invalid variable type: "<class 'netCDF4.VLType'>": string type (vlen    │
│        │            │          │ types not supported)                                                           │
│ Region │ cf         │ WARN     │ (3): No standard_name or long_name attribute specified                         │
│ lat    │ ferret     │ ERROR    │ Coordinate 'lat' has forbidden _FillValue (attrs=np.float64(nan)).             │
│ lon    │ ferret     │ ERROR    │ Coordinate 'lon' has forbidden _FillValue (attrs=np.float64(nan)).             │
└────────┴────────────┴──────────┴────────────────────────────────────────────────────────────────────────────────┘

                                                 Variable Findings                                                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope     ┃ Convention ┃ Severity ┃ Detail                                                                      ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fco2atm   │ cf         │ ERROR    │ (3.3): Invalid standard_name: Atmospheric                                   │
│ fco2atm   │ cf         │ ERROR    │ (3.3): Invalid standard_name modifier: fCO2                                 │
│ fgco2_reg │ cf         │ ERROR    │ (3.1): Invalid units: PgC yr-1                                              │
│ sfco2     │ cf         │ ERROR    │ (3.3): Invalid syntax for 'standard_name' attribute: 'Sea surface fCO2'     │
│ tos       │ cf         │ ERROR    │ (3.3): Invalid syntax for 'standard_name' attribute: 'Sea surface           │
│           │            │          │ temperature'                                                                │
└───────────┴────────────┴──────────┴─────────────────────────────────────────────────────────────────────────────┘

                                              Suggested Improvements                                               
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Variable  ┃ Suggestion                                                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fgco2     │ {'recommended_standard_names': ['ocean_volume', 'sea_area', 'air_pressure_at_sea_level'],           │
│           │ 'recommended_units': 'm3', 'domain': 'ocean'}                                                       │
│ sfco2     │ {'unknown_standard_name': 'Sea surface fCO2'}                                                       │
│ fco2atm   │ {'unknown_standard_name': 'Atmospheric fCO2'}                                                       │
│ kw        │ {'recommended_standard_names': ['direction_of_sea_ice_velocity', 'direction_of_sea_water_velocity', │
│           │ 'divergence_of_sea_ice_velocity'], 'recommended_units': 'degree', 'domain': 'ocean'}                │
│ alpha     │ {'recommended_standard_names': ['mass_concentration_of_alpha_carotene_in_sea_water',                │
│           │ 'concentration_of_chlorophyll_in_sea_water', 'speed_of_sound_in_sea_water'], 'recommended_units':   │
│           │ 'kg m-3', 'domain': 'ocean'}                                                                        │
│ SIC       │ {'units_check': {'status': 'mismatch', 'current_units': 'percent', 'expected_units': '1'}}          │
│ area      │ {'recommended_standard_names': ['sea_area', 'sea_water_mass_per_unit_area', 'sea_area_fraction'],   │
│           │ 'recommended_units': 'm2', 'domain': 'ocean'}                                                       │
│ tos       │ {'unknown_standard_name': 'Sea surface temperature'}                                                │
│ fgco2_reg │ {'recommended_standard_names': ['ocean_volume', 'sea_area', 'air_pressure_at_sea_level'],           │
│           │ 'recommended_units': 'm3', 'domain': 'ocean'}                                                       │
│ mask_sfc  │ {'recommended_standard_names': ['area_fraction', 'sea_area_fraction',                               │
│           │ 'area_fraction_below_surface'], 'recommended_units': '1', 'domain': 'ocean'}                        │
│ area_reg  │ {'recommended_standard_names': ['sea_area', 'sea_area_fraction', 'sea_ice_area'],                   │
│           │ 'recommended_units': 'm2', 'domain': 'ocean'}                                                       │
└───────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────┘

## 4. Apply compliance fixes


In [31]:
fixed = ds.check.make_cf_compliant()
fixed["temperature"].attrs["standard_name"] = "sea_surface_temperature"
fixed["temperature"].attrs["units"] = "degC"

print(type(fixed))
print("Conventions:", fixed.attrs.get("Conventions"))
print("lat attrs:", fixed["lat"].attrs)
print("lon attrs:", fixed["lon"].attrs)
print("time attrs:", fixed["time"].attrs)

fixed

<class 'xarray.core.dataset.Dataset'>
Conventions: CF-1.12
lat attrs: {'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_north', 'axis': 'Y'}
lon attrs: {'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_east', 'axis': 'X'}
time attrs: {'standard_name': 'time', 'axis': 'T'}


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8052 0.5414 ... 0.6556 0.5747
Attributes:
    Conventions:  CF-1.12

## 5. Re-check after coercion


In [32]:
issues_after = fixed.check.check_compliant(pretty_print=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ CF Compliance Report                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CF version     CF-1.12               
Engine         cfchecker             
Engine status  unavailable           
Counts         fatal=0 error=0 warn=0

          Coordinate Findings          
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Scope ┃ Severity ┃ Detail           ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ time  │ INFO     │ coord_attr:units │
└───────┴──────────┴──────────────────┘

                                         Suggested Improvements                                          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Variable    ┃ Suggestion                                                                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ temperature │ {'units_check': {'status': 'mismatch', 'current_units': 'degC', 'expected_units': 'K'}} │
└─────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                             Notes                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Note                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ cfchecker could not run; returned heuristic checks instead. │
└─────────────────────────────────────────────────────────────┘

╭───────────────────────────────────────────────── Checker Error ─────────────────────────────────────────────────╮
│ {'type': 'AttributeError', 'message': 'NetCDF: Attribute not found'}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 6. Other dimension patterns


In [33]:
# time-only
only_time = xr.Dataset(
    data_vars={"signal": (("time",), [1.0, 2.0, 3.0])},
    coords={"time": [0, 1, 2]},
)

# lat-lon only
only_space = xr.Dataset(
    data_vars={"field": (("lat", "lon"), np.ones((2, 2)))},
    coords={"lat": [0.0, 1.0], "lon": [10.0, 11.0]},
)

print(
    "time-only conventions:",
    only_time.check.make_cf_compliant().attrs.get("Conventions"),
)
print(
    "space-only conventions:",
    only_space.check.make_cf_compliant().attrs.get("Conventions"),
)

time-only conventions: CF-1.12
space-only conventions: CF-1.12


## 7. Write to NetCDF (optional)

Once compliant, you can write normally:

```python
fixed.to_netcdf("example_cf.nc")
```
